<a href="https://colab.research.google.com/github/jonatastbelotti/mo432-trab2/blob/master/Trab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jônatas Trabuco Belotti</br>
RA: 230260</br>
jonatas.t.belotti@hotmail.com

# Leia
Leia o arquivo Bias_correction_ucl.csv. O atributo de saída é Next_Tmax (a temperatura máxima no próximo dia). A derscriçao dos dados esta [aqui](https://archive.ics.uci.edu/ml/datasets/Bias+correction+of+numerical+prediction+model+temperature+forecast):


* Remova a coluna Date;
* Remova a coluna Next_Tmin (coluna 25). O objetivo original dos dados é prever tanto Next_Tmin quanto Next_tmax. Mas nos so vamos usar o Next_Tmax. Mas não queremos que o valor do Next_Tmin influencie nessa previsão;
* remova as linhas que tem valor faltante. Das 7752 linhas originais sobram 7588.



In [1]:
import pandas as pd

df = pd.read_csv("/content/Bias_correction_ucl.csv")
df = df.drop(["Date", "Next_Tmin"], axis=1) # Remove colunas
df = df.dropna() # Remove toda linha com pelo menos um valor faltante
df

,station,Present_Tmax,Present_Tmin,LDAPS_RHmin,LDAPS_RHmax,LDAPS_Tmax_lapse,LDAPS_Tmin_lapse,LDAPS_WS,LDAPS_LH,LDAPS_CC1,LDAPS_CC2,LDAPS_CC3,LDAPS_CC4,LDAPS_PPT1,LDAPS_PPT2,LDAPS_PPT3,LDAPS_PPT4,lat,lon,DEM,Slope,Solar radiation,Next_Tmax
0,1.0,28.7,21.4,58.255688,91.116364,28.074101,23.006936,6.818887,69.451805,0.233947,0.203896,1.616969e-01,0.130928,0.0,0.0,0.0,0.0,37.6046,126.991,212.3350,2.7850,5992.895996,29.1
1,2.0,31.9,21.6,52.263397,90.604721,29.850689,24.035009,5.691890,51.937448,0.225508,0.251771,1.594441e-01,0.127727,0.0,0.0,0.0,0.0,37.6046,127.032,44.7624,0.5141,5869.312500,30.5
2,3.0,31.6,23.3,48.690479,83.973587,30.091292,24.565633,6.138224,20.573050,0.209344,0.257469,2.040915e-01,0.142125,0.0,0.0,0.0,0.0,37.5776,127.058,33.3068,0.2661,5863.555664,31.1
3,4.0,32.0,23.4,58.239788,96.483688,29.704629,23.326177,5.650050,65.727144,0.216372,0.226002,1.611574e-01,0.134249,0.0,0.0,0.0,0.0,37.6450,127.022,45.7160,2.5348,5856.964844,31.7
4,5.0,31.4,21.9,56.174095,90.155128,29.113934,23.486480,5.735004,107.965535,0.151407,0.249995,1.788925e-01,0.170021,0.0,0.0,0.0,0.0,37.5507,127.135,35.0380,0.5055,5859.552246,31.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7745,21.0,23.1,17.8,24.688997,78.261383,27.812697,18.303014,6.603253,9.614074,0.052902,0.030169,0.000000e+00,0.004380,0.0,0.0,0.0,0.0,37.5507,127.040,26.2980,0.5721,4456.024414,27.6
7746,22.0,22.5,17.4,30.094858,83.690018,26.704905,17.814038,5.768083,82.146707,0.066461,0.024518,5.850000e-07,0.017678,0.0,0.0,0.0,0.0,37.5102,127.086,21.9668,0.1332,4441.803711,28.0
7747,23.0,23.3,17.1,26.741310,78.869858,26.352081,18.775678,6.148918,72.058294,0.030034,0.081035,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,37.5372,126.891,15.5876,0.1554,4443.313965,28.3
7748,24.0,23.3,17.7,24.040634,77.294975,27.010193,18.733519,6.542819,47.241457,0.035874,0.074962,0.000000e+00,0.000000,0.0,0.0,0.0,0.0,37.5237,126.909,17.2956,0.2223,4438.373535,28.6


Separando as colunas do DataFrame em dados de entrada e dados de saída:

In [2]:
import numpy as np

entradas = df.drop(["Next_Tmax"], axis=1).to_numpy()
saidas = df["Next_Tmax"].to_numpy()
entradas.shape, saidas.shape

((7588, 22), (7588,))

# Execuções

Para cada um dos regressores abaixo aplique Validação Cruzada do tipo *5-fold*. Utilize RMSE como medida de erro de todos os regressores. A busca de hiperparametros será aleatória. O problema especificará um intervalo para hiperparametro. Use uma distribuição uniforme para escolher valores neste intervalo.

Reporte o valor do RMSE da melhor combinação de hiperparametros e o valor dos hiperparametros encontrados para cada regressor. Também reporte os valores do RMSE para os regressores do Scikit-learn com os valores padrão de hiperparâmetros.

## Regressão Linear
Não tem hiperparâmetro. Portanto, foi implementada apenas a validação cruzada.



In [35]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
import numpy as np

resultados = cross_validate(LinearRegression(), entradas, saidas, cv=5, scoring="neg_root_mean_squared_error")
resultados = resultados['test_score']
print("RMSE: %s" % str(resultados))
print("RMSE médio: %s" % str(np.mean(resultados)))

RMSE: [-1.45728827 -1.63874781 -1.45413013 -1.62441096 -1.7131539 ]
RMSE médio: -1.5775462124225534
